## Example 01 - Basic Hopper Operation

**Example description:** Example of a trailing suction hopper dredge shipping sediment from origin to destination site.

* [**0. Import required libraries:**](#0.-Import-required-libraries)<br>
* [**1. Define work method:**](#1.-Define-work-method)<br>
   * [**1.1 Sites:**](#1.1-Define-the-project-sites)<br>
   * [**1.2 Equipment:**](#1.2-Define-the-project-equipment)<br>
   * [**1.3 Activities:**](#1.3-Define-the-activity)<br>
* [**2. Run the simulation:**](#2.-Run-the-simulation)<br>
* [**3. Post processing:**](#3.-Postprocessing)<br>

## 0. Import required libraries

In [21]:
# package(s) related to time, space and id
import datetime, time
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np

# digital twin package
import digital_twin.core as core
import digital_twin.model as model
import digital_twin.plot as plot

# Additional import to save the initialization of the simulation
import digital_twin.savesim as savesim
import halem
import pickle

In [22]:
# Create simulation environment

t0 = '17/04/2019 01:00:00'
d = datetime.datetime.strptime(t0, "%d/%m/%Y %H:%M:%S")
t0 = d.timestamp()

simulation_start = datetime.datetime.fromtimestamp(t0)

my_env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
my_env.epoch = time.mktime(simulation_start.timetuple())

## 1. Define work method

### 1.1 Define the project sites
You can specify a project site object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.Locatable - enables you to add coordinates to extract distance information and visualize
    core.HasContainer - enables you to add information on the material available at the site
    core.HasResource - enables you to add information on serving equipment
    core.HasWeather - enables you to add weather conditions

#### First create objects with the desired properties

In [23]:
# The generic site class
Site = type('Site', (core.Identifiable, # Give it a name
                     core.Log,          # Allow logging of all discrete events
                     core.Locatable,    # Add coordinates to extract distance information and visualize
                     core.HasContainer, # Add information on the material available at the site
                     core.HasResource), # Add information on serving equipment
            {})                         # The dictionary is empty because the site type is generic

#### Next specify the properties for sites you wish to create

In [24]:
# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.788699, 52.970919)  # lon, lat

data_from_site = {"env": my_env,                                # The simpy environment defined in the first cel
                  "name": "Winlocatie",                         # The name of the site
                  "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5b", # For logging purposes
                  "geometry": location_from_site,               # The coordinates of the project site
                  "capacity": 30_000,                          # The capacity of the site
                  "level": 30_000}                             # The actual volume of the site

In [25]:
# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.541166, 53.093619)     # lon, lat

data_to_site = {"env": my_env,                                # The simpy environment defined in the first cel
                "name": "Dumplocatie",                        # The name of the site
                "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5b", # For logging purposes
                "geometry": location_to_site,                 # The coordinates of the project site
                "capacity": 30_000,                          # The capacity of the site
                "level": 0}                                   # The actual volume of the site (empty of course)

#### Finally create specific instances of the predefined objects with the specified properties

In [26]:
# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site   = Site(**data_to_site)

### 1.2 Define the project equipment
You can specify a vessel object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.ContainerDependentMovable - A moving container, so capacity and location
    core.Processor - Allow for loading and unloading
    core.HasResource - Add information on serving equipment
    core.HasDepthRestriction - Add information on depth restriction 

#### First create objects with the desired properties

In [27]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type('TransportProcessingResource', 
                                   (core.Identifiable,              # Give it a name
                                    core.Log,                       # Allow logging of all discrete events
                                    core.ContainerDependentMovable, # A moving container, so capacity and location
                                    core.Processor,                 # Allow for loading and unloading
                                    core.HasResource,               # Add information on serving equipment
                                    core.HasCosts,
                                    ),                 # Initialize spill terms
                                   {})

#### Next specify the properties for vessel(s) you wish to create

In [28]:
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty

def compute_loading(rate):
    return lambda current_level, desired_level: (desired_level - current_level) / rate

def compute_unloading(rate):
    return lambda current_level, desired_level: (current_level - desired_level) / rate

In [29]:
# TSHD variables
data_hopper = {"env": my_env,                                       # The simpy environment 
               "name": "Hopper 01",                                 # Name
               "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",        # For logging purposes
               "geometry": location_from_site,                      # It starts at the "from site"
               "loading_func": compute_loading(1.5),                # Loading rate
               "unloading_func": compute_unloading(1.5),            # Unloading rate
               "capacity": 5_000,                                   # Capacity of the hopper - "Beunvolume"
               "compute_v": compute_v_provider(7, 5),             # Variable speed 
               "weekrate": 700_000}

#### Finally create specific instances of the predefined objects with the specified properties

In [30]:
# The simulation object
hopper = TransportProcessingResource(**data_hopper)

### 1.3 Define the activity

In [31]:
# Create activity
activity = model.Activity(env = my_env,           # The simpy environment defined in the first cel
                          name = "Soil movement", # We are moving soil
                          ID = "6dbbbdf7-4589-11e9-bf3b-b469212bff5b", # For logging purposes
                          origin = from_site,     # We originate from the from_site
                          destination = to_site,  # And therefore travel to the to_site
                          loader = hopper,        # The benefit of a TSHD, all steps can be done
                          mover = hopper,         # The benefit of a TSHD, all steps can be done
                          unloader = hopper,      # The benefit of a TSHD, all steps can be done
                          start_event = None,     # We can start right away
                          stop_event = None)      # We stop once there is nothing more to move

In [32]:
name_textfile_load = '../tests/Roadmap/General_waddensea_dt=3h'

with open(name_textfile_load, 'rb') as input:
    Roadmap = pickle.load(input)\
    
print('One specific example of a roadmap is displayed in\
../Hydraulic-Infrastructure-Realisation/Roadmap/02_Road_map_generator(Noos_Matroos_Data).ipynb \
Different use cases need different Roadmaps need to be made.\
This Roadmap is valid for t∈[{},{}], with a timestep of 1 hour and a minimal waterdepth of 6m and a ship velocity of 5 or 7 m/s'
      .format(datetime.datetime.fromtimestamp(Roadmap.t[0]),datetime.datetime.fromtimestamp(Roadmap.t[-1])))

One specific example of a roadmap is displayed in../Hydraulic-Infrastructure-Realisation/Roadmap/02_Road_map_generator(Noos_Matroos_Data).ipynb Different use cases need different Roadmaps need to be made.This Roadmap is valid for t∈[2019-04-17 00:00:00,2019-04-18 00:00:00], with a timestep of 1 hour and a minimal waterdepth of 6m and a ship velocity of 5 or 7 m/s


### 2. Run the simulation

In [33]:
my_env.Roadmap = Roadmap
my_env.run()

print("")
print("*** Installation of dike finished in {}".format(datetime.timedelta(seconds=int(my_env.now - my_env.epoch))))
print("*** Installation cost {:,.2f}€.".format(int(hopper.cost)))


*** Installation of dike finished in 23:12:45
*** Installation cost 96,719.00€.


### 3. Postprocessing

#### Vessel planning

In [34]:
vessels = [hopper]

activities = ['loading', 'unloading', 'sailing filled', 'sailing empty']
colors = {0:'rgb(55,126,184)', 1:'rgb(255,150,0)', 2:'rgb(98, 192, 122)', 3:'rgb(98, 141, 122)'}

plot.vessel_planning(vessels, activities, colors)

### Save all logs

In [15]:
sites = [from_site, to_site]
equipment = [hopper]
activities = [activity]

save = savesim.LogSaver(sites, equipment, activities, 
                        simulation_id = '5e1d7c36-458e-11e9-816b-b469212bff5b', simulation_name = "Example 01",
                        location = "Simulation Results", overwrite = True)

In [16]:
plot.vessel_kml(my_env, vessels, stepsize = 600)
#plot.site_kml(my_env, [Nodes[0], Nodes[-1]], stepsize = 600)
# plot.graph_kml(my_env)

# open the file
if platform.system():
    !start ./vessel_movements.kml
    # !start ./site_development.kml
else:
    !start explorer ./vessel_movements.kml
    # !start explorer ./site_development.kml

In [37]:
import pandas as pd
pd.DataFrame.from_dict(hopper.log).head(50)

,Message,Timestamp,Value,Geometry
0,loading start,2019-04-17 01:00:00.000000,0.0,POINT (4.788699 52.970919)
1,loading start,2019-04-17 01:00:00.000000,5000.0,POINT (4.788699 52.970919)
2,loading stop,2019-04-17 01:55:33.333333,5000.0,POINT (4.788699 52.970919)
3,loading stop,2019-04-17 01:55:33.333333,5000.0,POINT (4.788699 52.970919)
4,sailing filled start,2019-04-17 01:55:33.333333,5000.0,POINT (4.788699 52.970919)
5,Sailing,2019-04-17 01:55:33.333333,0.0,POINT (4.788699 52.970919)
6,Sailing,2019-04-17 02:00:04.632936,0.0,POINT (4.787599563598633 52.97394561767578)
7,Sailing,2019-04-17 02:04:38.763191,0.0,POINT (4.772323131561279 52.97868728637695)
8,Sailing,2019-04-17 02:09:04.316915,0.0,POINT (4.756532192230225 52.9819450378418)
9,Sailing,2019-04-17 02:13:57.868216,0.0,POINT (4.740272521972656 52.98615264892578)


In [38]:
hopper.log.keys()

dict_keys(['Message', 'Timestamp', 'Value', 'Geometry'])